In [1]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from keras.optimizers import SGD, Adam
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv('C:\\Users\\aarus\\Downloads\\archive\\A_Z Handwritten Data.csv').astype('int8')

In [ ]:
df=df[0:250000]

In [3]:
shuffled_df = df.sample(frac=1, random_state=42)  # Shuffle the DataFrame

In [4]:
shuffled_df.head()


,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648
114871,12,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
270469,18,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27001,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180911,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
label = {0:'A',1:'B',2:'C',3:'D',4:'E',5:'F',6:'G',7:'H',8:'I',9:'J',10:'K',11:'L',12:'M',13:'N',14:'O',15:'P',16:'Q',17:'R',18:'S',19:'T',20:'U',21:'V',22:'W',23:'X',24:'Y',25:'Z'}
shuffled_df.iloc[:,0] = shuffled_df.iloc[:,0].map(label)

shuffled_df.head(10)

In [ ]:
shuffled_df = df.sample(frac=1).reset_index(drop=True)

In [5]:
df.nunique().count()

785

In [ ]:
shuffled_df.head(3)

In [6]:
df=df.rename(columns={'0': 'label'})

In [7]:
df.sample(5)

,label,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.639,0.640,0.641,0.642,0.643,0.644,0.645,0.646,0.647,0.648
58783,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88168,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
301433,19,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
209570,15,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
140071,14,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
X = df.drop('label', axis=1).values
y = df['label'].values

X = X.reshape(-1, 28*28)

y.shape

In [ ]:
X.shape

In [9]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size = 0.2)

train_x = np.reshape(train_x, (train_x.shape[0], 28,28))
test_x = np.reshape(test_x, (test_x.shape[0], 28,28))

print("Train data shape: ", train_x.shape)
print("Test data shape: ", test_x.shape)

Train data shape:  (297960, 28, 28)
Test data shape:  (74490, 28, 28)


In [10]:
train_X = train_x.reshape(train_x.shape[0],train_x.shape[1],train_x.shape[2],1)
print("New shape of train data: ", train_X.shape)

test_X = test_x.reshape(test_x.shape[0], test_x.shape[1], test_x.shape[2],1)
print("New shape of train data: ", test_X.shape)

#Now we reshape the train & test image dataset so that they can be put in the model.


New shape of train data:  (297960, 28, 28, 1)
New shape of train data:  (74490, 28, 28, 1)


In [11]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28,28,1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding = 'same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding = 'valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))

model.add(Flatten())

model.add(Dense(64,activation ="relu"))
model.add(Dense(128,activation ="relu"))

model.add(Dense(26,activation ="softmax"))

In [12]:
train_yOHE = to_categorical(train_y, num_classes = 26, dtype='int')
print("New shape of train labels: ", train_yOHE.shape)

test_yOHE = to_categorical(test_y, num_classes = 26, dtype='int')
print("New shape of test labels: ", test_yOHE.shape)

New shape of train labels:  (297960, 26)
New shape of test labels:  (74490, 26)


In [ ]:
model.compile(optimizer = Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_X, train_yOHE, epochs=1,  validation_data = (test_X,test_yOHE))

In [20]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_X, train_yOHE, epochs=2, batch_size=32, validation_data=(test_X,test_yOHE))

# Evaluate the model on the test set


Epoch 1/2
9312/9312 [==============================] - 252s 27ms/step - loss: 0.2030 - accuracy: 0.9407 - val_loss: 0.1329 - val_accuracy: 0.9618
Epoch 2/2
2328/2328 [==============================] - 28s 12ms/step


NameError: name 'accuracy_score' is not defined

TypeError: 'numpy.dtype[float32]' object is not callable

In [22]:
predictions = model.predict(test_X)

2328/2328 [==============================] - 26s 11ms/step


In [26]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(test_y, np.argmax(predictions, axis=1))
print(f'Test accuracy: {accuracy}')

Test accuracy: 0.9629346220969257


In [27]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 6, 6, 64)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 2, 2, 128)        0

In [21]:
model.save(r'model_hand1.h5')

In [ ]:
import cv2
import numpy as np
import tensorflow as tf


model = tf.keras.models.load_model("C:\\Users\\aarus\\model_hand1.h5")


canvas_size = 200
canvas = np.ones((canvas_size, canvas_size), dtype=np.uint8)
drawing = False


def draw(event, x, y, flags, param):
    global canvas, drawing
    
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        cv2.circle(canvas, (x, y), 10, 255, -1)  # Draw a white circle on click
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.circle(canvas, (x, y), 10, 255, -1)
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

# Set up OpenCV window
cv2.namedWindow('Draw Alphabet')
cv2.setMouseCallback('Draw Alphabet', draw)

while True:
    cv2.imshow('Draw Alphabet', canvas)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):  # Press 'q' to quit
        break
    elif key == ord('p'):  # Press 'p' to predict
        # Preprocess drawn image
        resized_img = cv2.resize(canvas, (28, 28))
        normalized_img = resized_img / 255.0
        input_img = np.expand_dims(normalized_img, axis=0)
        
        # Make prediction
        predictions = model.predict(input_img)
        predicted_class = np.argmax(predictions)
        predicted_alphabet = chr(ord('A') + predicted_class)
        
        print(f'Predicted alphabet: {predicted_alphabet}')
        
        result_display = np.ones((canvas_size, canvas_size), dtype=np.uint8) * 255
        cv2.putText(result_display, f'Predicted: {predicted_alphabet}', (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.8, 0, 2)
        cv2.imshow('Draw Alphabet', result_display)

cv2.destroyAllWindows()

1/1 [==============================] - 0s 78ms/step
Predicted alphabet: O


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
X_train.shape, y_train.shape,X_test.shape, y_test.shape

In [ ]:
model=RandomForestClassifier()

In [ ]:
train_X=X_train[0:180000]
train_Y=y_train[0:180000]

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_pred[0]

In [ ]:
model.predict(X_test[0].reshape(1, -1))

In [ ]:
acc= accuracy_score(y_test, y_pred)

In [ ]:
acc

In [ ]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)
